<a href="https://colab.research.google.com/github/ratulb/pytorch/blob/main/mnist_in_pythor_from_scratch_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = "https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/tabular-wine/winequality-white.csv"
!curl -O $url

In [ ]:
import csv


In [ ]:
wine_path="/content/winequality-white.csv"

wineq_data = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
columns = next(csv.reader(open(wine_path), delimiter=";"))
wineq_data.shape, columns
wineq = torch.from_numpy(wineq_data)
wineq.shape, wineq.type()

In [ ]:
data = wineq[:, :-1]
data, data.shape


In [ ]:
target = wineq[:, -1]
target, target.shape

In [ ]:
from PIL import Image
import numpy as np
img = Image.open("/content/dog.png")
print(type(img))
image_arr = torch.from_numpy(np.array(img))
print(image_arr.shape)
img

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image_tensor = img_t.permute(1, 2, 0)

# Convert the tensor to a NumPy array
image_array = image_tensor.numpy()

# Display the image
plt.imshow(image_array)
plt.axis('off')  # Hide axes for better visualization
plt.show()

In [ ]:
import random

# Initialize weights and biases
random.seed(142)
weights = [random.random() for x in range(9)]
weights = [[x * 10 for x in weights[:3]], [x * 10 for x in weights[3:6]], [x * 10 for x in weights[6:]]]
biases = [random.random() * 10 for _ in range(3)]

# Generate batch of inputs and corresponding targets
batch_size = 4
inputs_batch = [[1.2, -2.6, 0.3], [0.5, 1.2, -1.3], [2.1, -1.9, 0.7], [0.8, -0.6, 0.2]]
targets_batch = [[-5, 7, 25], [3, -2, 0], [10, 5, -1], [-7, 8, 3]]

learning_rate = 0.01
num_epochs = 100

def forward(current_batch, current_layers_neurons_weights, neurons_biases):
    """
    Compute the output of the current layer for each sample in the batch.

    Args:
        current_batch (list): A list of samples in the batch.
        current_layers_neurons_weights (list): A list of weights for the neurons in the current layer.
        neurons_biases (list): A list of biases for the neurons in the current layer.

    Returns:
        batch_outputs (list): A list where each entry corresponds to the output of the current layer
                                for a sample in the batch. Each entry is a list of neuron outputs.
                                If the layer has 3 neurons, each entry will have a length of 3.
                                For example, if the current batch contains 1 sample and the layer has 3 neurons,
                                the output will be:
                                [[output of n1, output of n2, output of n3]]
                                If the batch contains 2 samples, the output will be:
                                [[output of n1, output of n2, output of n3],
                                 [output of n1, output of n2, output of n3]]
                                Each inner list represents the neuron outputs for a corresponding sample in the batch.
    """
    batch_outputs = []

    for each_sample in current_batch:
        this_sample = each_sample
        outputs_of_neurons = [0] * len(current_layers_neurons_weights)
        for this_neuron, its_weights in enumerate(current_layers_neurons_weights):
            bias_of_this_neuron = neurons_biases[this_neuron]
            outputs_of_neurons[this_neuron] = bias_of_this_neuron  # Start with the bias
            for feature_index, feature in enumerate(this_sample):
                outputs_of_neurons[this_neuron] += its_weights[feature_index] * feature
        batch_outputs.append(outputs_of_neurons)
    return batch_outputs

def batched_errors(batch_outputs, batch_targets):
    batch_errors = []
    for outputs, targets in zip(batch_outputs, batch_targets):
        sample_errors = []
        for output, target in zip(outputs, targets):
            residual = (output - target)
            squared_error = residual ** 2
            sample_errors.append((squared_error, residual))
        batch_errors.append(sample_errors)
    return batch_errors

def weight_deltas(batch_errors, inputs_batch):
    batch_weight_deltas = []
    bias_deltas = [0] * len(batch_errors[0])  # Initialize bias deltas to zero
    for errors, inputs in zip(batch_errors, inputs_batch):
        sample_weight_deltas = []
        for neuron, error in enumerate(errors):
            weight_delta = [x * error[1] * learning_rate for x in inputs]
            sample_weight_deltas.append(weight_delta)
            bias_deltas[neuron] += error[1] * learning_rate  # Accumulate bias delta
        batch_weight_deltas.append(sample_weight_deltas)
    return batch_weight_deltas, bias_deltas

def update_weights(weights, biases, batch_weight_deltas, bias_deltas):
    for weight_deltas in batch_weight_deltas:
        for i in range(len(weights)):
            for j in range(len(weights[i])):
                weights[i][j] -= weight_deltas[i][j]
    for i in range(len(biases)):
        biases[i] -= bias_deltas[i] / len(batch_weight_deltas)  # Average the bias updates
    return weights, biases

# Training loop
for epoch in range(num_epochs):
    batch_preds = forward(inputs_batch, weights, biases)
    batch_errors_ = batched_errors(batch_preds, targets_batch)
    batch_weight_deltas, bias_deltas = weight_deltas(batch_errors_, inputs_batch)
    weights, biases = update_weights(weights, biases, batch_weight_deltas, bias_deltas)

print("Updated weights:", weights)
print("Updated biases:", biases)
